<a href="https://colab.research.google.com/github/Souhardya-27/My-Project/blob/main/Heart_Failure_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
#Dependencies
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [42]:
np.random.seed(42)

Samples = 2000    #Number of patient
Time_steps = 30   #30 time points
Features = 8      #Vital signs + medical history
Batch_size = 32
epochs = 10
Device = "cuda" if torch.cuda.is_available() else "cpu"

In [43]:
def generate_patient():


  #Simulates one patient's continuous sensor stream.
  #Medical realism is injected using conditions like diabetes & hypertension.


  age = np.random.randint(18,80)
  diabetes = np.random.randint(0,2)
  hypertension = np.random.randint(0,2)
  heart_disease = np.random.randint(0,2)

  # Time-dependent vitals (sensor signals)
  hr = np.random.normal(80 + hypertension*15,15,Time_steps) #heart_rate
  sbp = np.random.normal(120 + hypertension*25,20,Time_steps) #systolic_blood_pressure
  dbp = np.random.normal(80 + hypertension*15,20,Time_steps)  #diastolic_blood_pressure
  spo2 = np.random.normal(97 - diabetes*5,2,Time_steps) #spo2
  ecg = np.random.randint(0,1,Time_steps) #ecg

  #static features repeated across time
  age_arr = np.full(Time_steps,age)
  diab_arr = np.full(Time_steps,diabetes)
  hyp_arr = np.full(Time_steps,hypertension)
  heart_arr = np.full(Time_steps,heart_disease)

  #combine all features -> shape(Time_steps,Features)
  series = np.vstack([age_arr,hr,sbp,dbp,spo2,ecg,diab_arr,hyp_arr]).T

  return series


In [44]:
#Generate dataset
X = np.array([generate_patient() for _ in range(Samples)])

**Medical Logic**

In [45]:
def risk_label(x):
  if np.mean(x[:,1]) > 120 or np.mean(x[:,4]) < 90 or np.mean(x[:,5]) > 0.75:
    return 2  #High Risk
  elif np.mean(x[:,1]) > 100 or np.mean(x[:,4]) < 150:
    return 1  #Medium Risk
  else:
    return 0  #Low Risk


y_risk = np.array([risk_label(x) for x in X])

In [46]:
def time_to_failure(risk):
  if risk == 2:
    return np.random.randint(1,2)     #1 months
  elif risk == 1:
    return np.random.randint(3,6)     #3-5 months
  else:
    return np.random.randint(12,24)   #safe range

y_time = np.array([time_to_failure(r) for r in y_risk])

y_risk_cat = tf.keras.utils.to_categorical(y_risk,3)

**Normalization**

In [47]:
scaler = StandardScaler()

X_scaled = X.reshape(-1,Features)
X_scaled = scaler.fit_transform(X_scaled)
X = X_scaled.reshape(Samples,Time_steps,Features)

**Dataset and Dataloader**

In [48]:
class HeartDataset(Dataset):
  def __init__(self,X,y_risk,y_time):
    self.X = torch.tensor(X,dtype = torch.float32)
    self.y_risk = torch.tensor(y_risk,dtype = torch.long)
    self.y_time = torch.tensor(y_time,dtype = torch.float32)

  def __len__(self):
    return len(self.X)

  def __getitem__(self,idx):
    return self.X[idx],self.y_risk[idx],self.y_time[idx]

dataset = HeartDataset(X,y_risk,y_time)
loader = DataLoader(dataset,batch_size=Batch_size,shuffle=True)

**Multi task LSTM Model**

In [49]:
class HeartFailureLSTM(nn.Module):
  def __init__(self,input_dim,hidden_dim,layer_dim,output_dim):
    super().__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim

    self.lstm = nn.LSTM(input_dim,hidden_dim,layer_dim,batch_first=True)
    self.fc_risk = nn.Linear(hidden_dim,output_dim)
    self.fc_time = nn.Linear(hidden_dim,1)

  def forward(self,x):
    h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
    c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)

    out,_=self.lstm(x,(h0,c0))
    out = out[:, -1, :]

    risk_logits = self.fc_risk(out)
    time_pred = self.fc_time(out)

    return risk_logits,time_pred

model = HeartFailureLSTM(input_dim=Features,hidden_dim=128,layer_dim=2,output_dim=3).to(Device)

**Loss + Optimizer**

In [50]:
criterion_risk = nn.CrossEntropyLoss()
criterion_time = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

**Training Loop**

In [51]:
for epoch in range(epochs):
  model.train()
  total_loss = 0

  for Xb,yr,yt in loader:
    Xb,yr,yt = Xb.to(Device),yr.to(Device),yt.to(Device)

    risk_logits,time_pred = model(Xb)

    loss_risk = criterion_risk(risk_logits,yr)
    loss_time = criterion_time(time_pred,yt)
    loss = loss_risk + loss_time

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"epoch [{epoch+1}/{epochs}] | Loss : {total_loss : .4f}")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch [1/10] | Loss :  238.7962
epoch [2/10] | Loss :  42.4494
epoch [3/10] | Loss :  42.1812
epoch [4/10] | Loss :  42.1739
epoch [5/10] | Loss :  42.3350
epoch [6/10] | Loss :  42.4758
epoch [7/10] | Loss :  42.7127
epoch [8/10] | Loss :  42.7480
epoch [9/10] | Loss :  42.2068
epoch [10/10] | Loss :  41.9663


**Real time device function**

In [52]:
def heart_failure_device(sensor_series):
  sensor_series = scaler.transform(sensor_series.reshape(-1, Features))
  sensor_series = torch.tensor(sensor_series, dtype=torch.float32).unsqueeze(0).to(Device)

  model.eval()
  with torch.no_grad():
    risk_logits, time_pred = model(sensor_series)

  risk = torch.argmax(risk_logits).item()
  months = int(time_pred.item())

  if risk == 2:
      msg = "🚨 HIGH RISK: Immediately start treatment!"
  elif risk == 1:
      msg = "⚠️ MEDIUM RISK: Take precautions"
  else:
      msg = "✅ LOW RISK: Continue monitoring"

  if sensor_series[0, -1, 1] > 150 and sensor_series[0, -1, 4] < 88:
      msg = "🆘 HEART ATTACK DETECTED! SAVE THE PATIENT IMMEDIATELY!"

  return {
      "Risk_Level": ["LOW", "MEDIUM", "HIGH"][risk],
      "Time_To_Failure": f"{months} months",
      "Message": msg
  }

**Live Simulation**

In [53]:
print("\n--- LIVE HEART MONITORING ---\n")
live_patient = generate_patient()
output = heart_failure_device(live_patient)
print(output)


--- LIVE HEART MONITORING ---

{'Risk_Level': 'MEDIUM', 'Time_To_Failure': '3 months', 'Message': '⚠️ MEDIUM RISK: Take precautions'}
